# NIRISS SOSS simulation of transiting exoplanet WASP-43b

This notebook demonstrates the core functionalities of the `mirage.soss_simulator` module by generating a simulation of the transiting exoplanet WASP-43b.

First, some imports...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import batman
from astropy.io import fits
import astropy.units as q
from mirage.soss_simulator import SossSim

## 1. Stellar Data
---------------

Let's load a theoretical spectrum of WASP-43 from file and plot it.

In [ ]:
# Load the spectrum
teff = 4250
logg = 4.6
feh = -0.01
jmag = 8
star_wavelength, star_flux = np.load('niriss_soss_example_data/WASP-43_Jmag_8.npy')
star_spectrum = [star_wavelength * q.um, star_flux * q.erg / q.s / q.cm**2 / q.AA]

# Make the plot
plt.plot(star_wavelength, star_flux)
plt.xlim(0.4,3.)
plt.xlabel('Wavelength (microns)')
plt.ylabel('Flux (erg/s/cm$^2$/angstrom)')
plt.title('Stellar spectrum of WASP-43')

## 2. Planet Data
---------------

Let's define a flat transit spectrum for WASP-43b.

In [ ]:
# Transmission spectrum
planet_tspectrum = [star_wavelength * q.um, np.ones_like(star_wavelength) * 0.0136059394]

And now let's generate a transit model `tmodel` and its parameters `params` using `batman` so `mirage` can inject those into the PSFs themselves. 

We first get the properties of WASP-43b from Kreidberg et al. (2014), and inject those in the `params` object. Let's assume we will be observing the transit of WASP-43b which occurrs on `2459819.131474` (August 27, 2022 --- at about 15:09 UT; note the `yaml` file is set to observe around this time indeed): 

In [ ]:
# Transit model with batman
params = batman.TransitParams()
params.t0 = 2459819.131474
params.per = 0.81347436
params.a = 4.87
params.inc = 82.10
params.ecc = 0.
params.w = 90.
params.limb_dark = 'quadratic'
params.u = [0.5,0.5]   # These are placeholders; they are changed internally
params.rp = 0.1166445  # These are placeholders; they are changed internally

Now create the `TransitModel` object; inject stellar info as well here. This is useful for calculating limb-darkening:

In [ ]:
tmodel = batman.TransitModel(params, np.array([0.])) # Time array is placeholder, will be changed internally by MIRAGE
tmodel.teff, tmodel.logg, tmodel.feh = teff, logg, feh

## 3. Run `mirage` with `SossSim`

All right, now we ingest all this to `mirage` --- if you don't have `exoctk`, this will set limb-darkening to zero (which is what we want for now I believe!):

In [ ]:
sim = SossSim(paramfile='niriss_soss_example_data/niriss_soss_substrip256_clear.yaml', \
              star=star_spectrum, planet=planet_tspectrum, tmodel=tmodel)

In [ ]:
sim.info

And, let's create the exposure! This should take a while:

In [ ]:
sim.create()

Let's read in the simulated data and plot one integration:

In [ ]:
# Get data from SCI extension
w43 = fits.getdata('WASP43_NIS_SOSS_CLEAR_uncal.fits')

# Plot a frame
plt.figure(figsize=(15,5))
im = plt.imshow(w43[0,2,:,:])
im.set_clim(10000,20000)